In [180]:
from pyvis.network import Network
import pandas as pd
import json

In [181]:
 # Opening JSON files
with open('../api/devices/phydot11_accesspoints.json', 'r') as openfile:
    aps = json.load(openfile)

with open('../api/phy_phy80211_ssids_views_ssids.json', 'r') as openfile:
    ssids = json.load(openfile)

with open('../api/devices/phy-IEEE802.11.json', 'r') as openfile:
    wlan = json.load(openfile)

In [182]:
aps_df = pd.DataFrame()

for ap in aps:    
    ap_data = {
        "key": [ap['kismet.device.base.key']], 
        "manuf": [ap['kismet.device.base.manuf']], 
        "macaddr": [ap['kismet.device.base.macaddr']]
    }
    
    aps_df = pd.concat([aps_df, pd.DataFrame(ap_data)], ignore_index=True)

In [183]:
wlan_df = pd.DataFrame()

for device in wlan:    
    wlan_data = {
        "key": [device['kismet.device.base.key']], 
        "manuf": [device['kismet.device.base.manuf']], 
        "macaddr": [device['kismet.device.base.macaddr']]
    }
    
    wlan_df = pd.concat([wlan_df, pd.DataFrame(wlan_data)], ignore_index=True)

In [184]:
ssids_df = pd.DataFrame()
ssids_num = pd.DataFrame()
columns = ['ssid', 'macaddr', 'manuf', 'weight', 'ap_bool']

for ssid in ssids:
    # ssid device numbers
    ssid_data = {
        "ssid": [ssid['dot11.ssidgroup.ssid']], 
        "num_probing": [ssid['dot11.ssidgroup.probing_devices_len']], 
        "num_responding": [ssid['dot11.ssidgroup.responding_devices_len']]
    }
    
    ssids_num = pd.concat([ssids_num, pd.DataFrame(ssid_data)], ignore_index=True)

    temp_res = pd.DataFrame(columns=columns)
    if (len(ssid['dot11.ssidgroup.responding_devices']) != 0):
        # responding devices devices nodes/edges
        temp_res = pd.DataFrame(ssid['dot11.ssidgroup.responding_devices'])
        temp_res['ssid'] = ssid['dot11.ssidgroup.ssid']
        temp_res.rename(columns={0: 'devices'}, inplace=True)
        temp_res = temp_res[['ssid', 'devices']]
        temp_res['weight'] = 2
        temp_res['ap_bool'] = True
        # Updating device keys with mac addresses
        temp_res = pd.merge(temp_res, aps_df, left_on='devices', right_on='key', how='inner')
        temp_res = temp_res[columns]
    
    temp_prob = pd.DataFrame(columns=columns)
    if (len(ssid['dot11.ssidgroup.probing_devices']) != 0):
        # probing devices nodes/edges
        temp_prob = pd.DataFrame(ssid['dot11.ssidgroup.probing_devices'])
        temp_prob['ssid'] = ssid['dot11.ssidgroup.ssid']
        temp_prob.rename(columns={0: 'devices'}, inplace=True)
        temp_prob = temp_prob[['ssid', 'devices']]
        temp_prob['weight'] = 1
        temp_prob['ap_bool'] = False
        # Updating device keys with mac addresses
        temp_prob = pd.merge(temp_prob, wlan_df, left_on='devices', right_on='key', how='inner')
        temp_prob = temp_prob[columns]

    # concat all
    ssids_df = pd.concat([ssids_df, temp_adv, temp_prob], ignore_index=True)

In [185]:
ssids_df['manuf'].value_counts()

manuf
Apple                              40
Unknown                            34
Intel Corporation                  10
Liteon Technology Corporation       1
Netgear                             1
TP-Link Technologies Ltd            1
TRENDnet Inc.                       1
Nintendo Ltd                        1
HP Inc.                             1
Chongqing Fugui Electronics Ltd     1
Name: count, dtype: int64

# graph viz

In [208]:
net = Network(
    notebook=True,
    cdn_resources="remote",
    height="750px", 
    width="100%", 
    bgcolor="#222222", 
    font_color="white",
    # select_menu=True,
    # filter_menu=True,
)

# Physics
net.force_atlas_2based()
# net.repulsion(
#     # node_distance=100,
#     # central_gravity=0.1,
#     spring_length=200,
#     # spring_strength=0.05,
#     damping=0.09,
# )
# net.show_buttons(filter_='physics')

sources = ssids_df['ssid']
targets = ssids_df['macaddr']
weights = ssids_df['weight']
manuf = ssids_df['manuf']
ap_bool = ssids_df['ap_bool']

edge_data = zip(sources, targets, weights, manuf, ap_bool)

images = {'Intel Corporation':'https://raw.githubusercontent.com/anrath/data/master/intel_logo.jpg', 'Apple':'https://raw.githubusercontent.com/anrath/data/master/apple_logo.png'}

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
    manuf = e[3]
    ap_bool = e[4]

    net.add_node(src, src, title=src, group=1, shape='diamond')

    device_group = 2 if ap_bool else 3

    if (manuf in images.keys()):
        net.add_node(dst, dst, title=src, group=device_group, shape='circularImage', image=images[manuf], brokenImage='circle')
    else:
        net.add_node(dst, dst, title=dst, group=device_group, shape='circularImage', image='https://raw.githubusercontent.com/anrath/data/master/blank.png', brokenImage='circle')
    net.add_edge(src, dst)


In [209]:
neighbor_map = net.get_adj_list()

# add neighbor data to node hover data
for node in net.nodes:
    node["title"] += " Neighbors:\n" + "\n".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

net.prep_notebook()
net.show("ssid.html")

ssid.html
